In [13]:
import pandas as pd
import numpy as np


In [14]:
stock_data = pd.read_csv('cleaned_stocks_data.csv')

In [15]:
stock_sentiment = pd.read_csv('stock_sentiment.csv')

In [16]:
stock_data.shape, stock_sentiment.shape

((284, 7), (217, 2))

In [17]:
stock_data.info() , stock_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       284 non-null    object 
 1   Open       284 non-null    float64
 2   High       284 non-null    float64
 3   Low        284 non-null    float64
 4   Close      284 non-null    float64
 5   Adj Close  284 non-null    float64
 6   Volume     284 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 15.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217 entries, 0 to 216
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Date             217 non-null    object 
 1   sentiment_score  217 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.5+ KB


(None, None)

In [18]:
import pandas as pd


# Merge on Date
new_df = pd.merge(stock_sentiment, stock_data, on='Date', how='inner')






In [19]:
new_df.shape

(156, 8)

In [20]:
new_df.head()

,Date,sentiment_score,Open,High,Low,Close,Adj Close,Volume
0,2025-05-19,0.959734,207.91,209.48,204.26,208.78,208.78,45310800
1,2025-05-16,0.961058,212.36,212.57,209.77,211.26,211.26,54737900
2,2025-05-15,0.949249,210.95,212.96,209.54,211.45,211.45,45029500
3,2025-05-14,0.940153,212.43,213.94,210.58,212.33,212.33,49325800
4,2025-05-13,0.888783,210.43,213.40,209.00,212.93,212.93,51909300


In [23]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error

# Assuming df is your DataFrame with these columns:
# 'sentiment_score', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'

# 1. Sort by Date just in case
new_df = new_df.sort_values('Date')

# 2. Select features and target
features = ['sentiment_score', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
target = 'Close'

data = new_df[features].values

# 3. Scale features between 0 and 1 (important for LSTM)
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# 4. Create sequences of, say, 60 days to predict next day
def create_sequences(data, seq_length=60):
    X, y = [], []
    for i in range(seq_length, len(data)):
        X.append(data[i-seq_length:i])
        y.append(data[i, features.index(target)])
    return np.array(X), np.array(y)

SEQ_LEN = 60
X, y = create_sequences(data_scaled, SEQ_LEN)

# 5. Train-test split (e.g. last 20% for test)
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]


ImportError: DLL load failed while importing lapack_lite: The specified module could not be found.